# Dataset: Imagenet 1000

### - Use pretrained models in the Keras Applications for applications: prediction, fine-tuning
### - models: ResNet50, VGG16, VGG19
### - Reference: https://keras.io/api/applications/ 
 - imagenet 1000 class list: 
     [keras-imagenet_class_index.json](https://github.com/raghakot/keras-vis/blob/master/resources/imagenet_class_index.json),
     [Class ID-Class Name Table](https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/),                                  [clsidx_to_lables.txt](https://gist.github.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57)
 
 - imagenet-1k Dataset card: https://huggingface.co/datasets/imagenet-1k
 - image-net.org: https://www.image-net.org/update-mar-11-2021.php

In [ ]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras import applications 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt

In [ ]:
# in order to download models from huggingface, it is necessary to set the following proxy and ssl 
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# suwon
import os
os.environ['REQUESTS_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'
os.environ['HTTP_PROXY'] ='http://75.17.107.42:8080'
os.environ['HTTPS_PROXY'] ='http://75.17.107.42:8080'

In [ ]:
img_path = '../images/'

In [ ]:
# load image from url
import urllib
from io import BytesIO

def load_image_url(URL):
    with urllib.request.urlopen(URL) as url:
        img = keras.preprocessing.image.load_img(BytesIO(url.read()), target_size=(224, 224))

    return img
    # return image.img_to_array(img)

In [ ]:
model = ResNet50(weights='imagenet')

In [ ]:
mobilenetv1 = applications.MobileNet(weights='imagenet')

In [ ]:
mobilenetv1.summary()
keras.utils.plot_model(mobilenetv1, to_file="mobilenetv1.png", show_shapes=True )

In [ ]:
mobilenetv2 = applications.MobileNetV2(weights='imagenet')

In [ ]:
mobilenetv2.summary()
keras.utils.plot_model(mobilenetv2, to_file="mobilenetv2.png", show_shapes=True )

In [ ]:
# load from an image in the local directory
img_file = img_path + 'elephant.jpg'
img = image.load_img(img_file, target_size=(224, 224)) # Loads an image into PIL format
plt.imshow(img)
plt.axis('off')
x = image.img_to_array(img) # PIL image into numpy array 224x224x3 (float32)
x = np.expand_dims(x, axis=0) # batch form by adding axis: 224x224x3 into (1x224x224x3)
x = preprocess_input(x) # substract mean values in [R, G, B]

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

In [ ]:
img_url = 'https://d1bg8rd1h4dvdb.cloudfront.net/img/storypick/monamipet/2019/01/1811_pet_dog_pomeranian_m_01.jpg'
a_img = load_image_url(img_url)
plt.imshow(a_img)
plt.axis('off')

x = image.img_to_array(a_img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, to_file="resnet50.png", show_shapes=True )

### Extract features with VGG16
- include_top=False: makes the last pooling layer the output layer (just before last FCs and softmax layers)
- cf. [vgg16 ref code](https://github.com/fchollet/deep-learning-models/blob/master/vgg16.py)
- you may compare the model summary by setting include_top=False and include_top=False

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

# include_top=False: excludes last FCs and softmax layers 
model = VGG16(weights='imagenet', include_top=False) 

img_file = img_path + 'elephant.jpg'
img = image.load_img(img_file, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, to_file="vgg16.png", show_shapes=True )

In [ ]:
features.shape

### Extract features from an arbitrary intermediate layer with VGG19

In [ ]:
base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

img_file = img_path + 'elephant.jpg'
img = image.load_img(img_file, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)

In [ ]:
model.summary()

In [ ]:
block4_pool_features.shape

In [ ]:
keras.utils.plot_model(base_model, to_file="vgg19_base.png", show_shapes=True )

In [ ]:
keras.utils.plot_model(model, to_file="vgg19_block4.png", show_shapes=True )